In [ ]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
# Đường dẫn đến tập tin CSV
#file_path = 'C:\\Users\\LG\\Desktop\\BD_DQ\\data_dotquy.csv'
#file_path = 'C:\\Users\\Admin\\Desktop\\BD_DQ\\data_dotquy.csv'
file_path = 'C:\\Users\\ACER\\Desktop\\BD_DQ\\data_dotquy.csv'#Hien
# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv(file_path)
# Hiển thị DataFrame trong VS Code
print(df)

LÀM SẠCH và CHUẨN HÓA DL
- 1 giới tính: loại other
- 2 tuổi: Loại số thực ( vd: 0.1 , 1.56) rồi dùng kỹ thuật Age Grouping
- 3 Đã kết hôn: chuyển về 0 và một
- 4 kiểu làm việc(work_type)
    + kỹ thuật Hashing Encoding
    + kỹ thuật One-Hot Encoding
- 5 vùng sống: chuyển về 0 và một
- 6 BMI  
    + Loại bỏ các hàng hoặc cột có N/A rồi dùng kỹ thuật Binary Encoding
    + dùng kỹ thuật Simple Imputation rồi dùng kỹ thuật tiếp Min-Max Scaling
- 7 smoking_status
  + Loại bỏ các hàng hoặc cột có Unknown rồi dùng kỹ thuật tiếp Min-Max Scaling
  + kỹ thuật Frequency Encoding
- 8 avg_glucose_level (mg/dL)
    DÙNG KỸ THUẬT: Min-Max Scaling
  ******ĐIỀU KIỆN LÀ******
    Người bình thường:
     < 140
    Người Bị tiểu đường:
      >=140

In [ ]:
#Hien
#Kiem tra trung lap
duplicated_rows = df[df.duplicated(subset='id')]
print("So luong du lieu trung lap la:" , len(duplicated_rows))

#df = df.dropna()
#df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )

In [ ]:
#hien
#Kiểm tra dữ liệu có bị thiếu
df.isnull().sum()

In [ ]:
#Hien
# dùng knn điền dữ liệu thiếu vào BMI
# Tạo đối tượng KNN Imputer với số lượng hàng xóm là 3
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# Áp dụng imputer cho DataFrame, chỉ chọn các cột số
df[['age', 'bmi']] = imputer.fit_transform(df[['age', 'bmi']])

# Hiển thị DataFrame sau khi điền dữ liệu
print(df)

In [ ]:
# Kiểm tra lại đã dủ dữ liệu chưa
df['bmi'].isnull().sum()
df['bmi']

In [ ]:
#Hien
# Xóa cột mã định danh
df  = df.drop(columns=['id'])
df

In [ ]:
#Chau_Muc1
#Loại bỏ giới tính mang giá trị Other
mask = df['gender'].isin(['Male', 'Female'])
df = df[mask]
print(df)

In [ ]:
#Chau_Muc1
# gender: female 1 male 0
df.loc[:, 'gender'] = df['gender'].replace({'Female': 1, 'Male': 0})
print(df)

In [ ]:
#Chau_Muc5(Phần này phân công cho GIANG)
# Residence_type: urban 1 rural 0(can note)
#df.loc[:, 'Residence_type'] = df['Residence_type'].replace({'Urban': 1, 'Rural': 0})


In [ ]:
#Chau_Muc4(Phần này lấy code của bạn GIANG)
#chuẩn hóa dữ liệu field work_type theo kỹ thuật label encoding
#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()
#df.loc[:, 'work_type'] = le.fit_transform(df['work_type'])
#print(df)

In [ ]:
#Chi_Muc2
# Kiểm tra xem tuổi có phải là số nguyên hay không
df['Is_Whole_Number'] = df['age'].apply(lambda x: x.is_integer())

# Lọc các hàng mà tuổi không phải là số thực (chỉ giữ lại số nguyên)
df = df[df['Is_Whole_Number']]

# Xóa cột tạm thời 'Is_Whole_Number'
df = df.drop(columns=['Is_Whole_Number'])

df

In [ ]:
#Chi_Muc2
# Chuẩn hóa bằng Min-Max Scaling
# Calculate min and max values
X_min = min(df['age'])
X_max = max(df['age'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['age']]

# Gán lại dữ liệu vào cột ['avg_glucose_level']
df['age'] = scaled_data
df

In [ ]:
#Chi(điều chỉnh tuổi về min max)
# Định nghĩa các khoảng tuổi và các nhãn cho các nhóm tuổi
#bins = [0, 18, 35, 60, 100]
#labels = ['0', '1', '2', '3']

# Chia cột "Age" thành các nhóm tuổi
#df['Age Group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

#df

In [ ]:
#Chi_Muc3
#Chuyen trạng thái đã kết hôn về dạng 1 và chưa kết hôn về dạng 0
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df

In [ ]:
#Giang_Muc4(Gán thẳng giá trị vào work_type)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['work_type'] = label_encoder.fit_transform(df['work_type'])

# sử dụng chuẩn hóa bằng Min-Max Scaling chuan hoa df['work_type'] dang (0 1 2 3 4)
# Calculate min and max values
X_min = min(df['work_type'])
X_max = max(df['work_type'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['work_type']]
scaled_data
# Tạo ra trường dữ liệu mới work_type_2
df['work_type'] =scaled_data
df

In [ ]:
#Giang_Muc5
# Chuyen trạng thái thành thị về dạng 1 và nông thôn về dạng 0
df['Residence_type']=df['Residence_type'].replace({'Urban':1,'Rural':0})
mask = df['gender'] != 'Other'
df_filtered = df[mask]
print(df_filtered) 

In [ ]:
#Hien_Mục6
# chuẩn hóa bằng Min-Max Scaling
glucose_values = df['avg_glucose_level']

# Calculate min and max values
X_min = min(df['avg_glucose_level'])
X_max = max(df['avg_glucose_level'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['avg_glucose_level']]

# Gán lại dữ liệu vào cột ['avg_glucose_level']
df['avg_glucose_level'] = scaled_data
df


In [ ]:
#Hoang_Muc7
# Dữ liệu ban đầu
data = df['bmi']

# Min-Max Scaling
data_min = data.min()
data_max = data.max()

scaled_data = (data - data_min) / (data_max - data_min)

# Hiển thị dữ liệu đã chuẩn hóa
print(scaled_data)

In [ ]:
#Hoang_Muc7
df['bmi']=scaled_data
df

In [ ]:
#Hoang_Muc7
# Dữ liệu ban đầu
data = df['bmi']

# Chuyển đổi dữ liệu thành DataFrame
dff = pd.DataFrame(data, columns=['bmi'])
dff

In [ ]:
#Hoang_Muc7
#import numpy as np
#conditions = [
   # (dff['bmi'] < 18.5),
  #  (dff['bmi'] >= 18.5) & (dff['bmi'] <= 25),
    #(dff['bmi'] > 25)
#]

# Các giá trị tương ứng cho mỗi điều kiện
#df['underweight'] = np.where(dff['bmi'] < 18.5, 1, 0)
#df['normal'] = np.where((dff['bmi'] >= 18.5) & (dff['bmi'] <= 25), 1, 0)
#df['overweight'] = np.where(dff['bmi'] > 25, 1, 0)

# Hiển thị DataFrame với dữ liệu đã chuẩn hóa
#df

In [ ]:
#Binh_Muc8
df.dropna(how='all', inplace=True)
df.head()

In [ ]:
#Binh_Muc8
df = df[df['smoking_status'] != 'Unknown']
df

In [ ]:
#Binh_Muc8
map_smoking_status = {
    'never smoked': 0,
    'formerly smoked': 0.5,
    'smokes': 1
}

df['smoking_status'] = df['smoking_status'].map(map_smoking_status)

print(df)

In [ ]:
#hien
#Trực quan hóa dữ liệu bằng biểu đồ nhiệt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#numeric_df = df.select_dtypes(include=[np.number])

corr = df.corr()
# plt.figure(figsize=(15,10))
# plt.title("Bảng tương quan các thuộc tính", frontsize = 16)
# sns.heatmap(corr, annot= True, cmap="Blues")

plt.figure(figsize=(15, 10))
plt.title("Bảng tương quan các thuộc tính", fontsize=16)
sns.heatmap(corr, annot=True, cmap='Blues')
plt.show()

CÂN BẰNG DỮ LIỆU Ở CỘT "STROKE"

In [ ]:
# Hien
# Kết hợp cả Oversampling và Undersampling để cân bằng dữ liệu 
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


# Tách dữ liệu thành các biến đặc trưng (X) và biến mục tiêu (y)
X = df.drop('stroke', axis=1)
y = df['stroke']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo các đối tượng Oversampling và Undersampling
over = SMOTE(sampling_strategy=0.5, random_state=42)  # Tăng cường lớp thiểu số
under = RandomUnderSampler(sampling_strategy=0.8, random_state=42)  # Giảm lớp chiếm ưu thế

# Tạo Pipeline kết hợp Oversampling và Undersampling
pipeline = Pipeline(steps=[('o', over), ('u', under)])

# Áp dụng pipeline trên dữ liệu huấn luyện
X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)

# Kiểm tra kích thước của dữ liệu sau khi cân bằng
print("Kích thước dữ liệu huấn luyện sau khi cân bằng:")
print(f"X_resampled: {X_resampled.shape}")
print(f"y_resampled: {y_resampled.shape}")

# Xem số lượng mẫu trong mỗi lớp sau khi cân bằng
print("Số lượng mẫu trong từng lớp sau khi cân bằng:")
print(y_resampled.value_counts())

# Kết hợp lại X_resampled và y_resampled để tạo thành một DataFrame hoàn chỉnh
df = pd.DataFrame(X_resampled, columns=X.columns)
df['stroke'] = y_resampled

# Xem trước dữ liệu sau khi cân bằng
df

In [ ]:
# xem số người bị đột quỵ trong bảng
dfff = df[df['stroke'] == 1]
print("Số người bị đột quỵ là " ) 
len(dfff)

In [ ]:
# xem số người KHÔNG bị đột quỵ trong bảng
dfff = df[df['stroke'] == 0]
print("Số người KHÔNG bị đột quỵ là " ) 
len(dfff)

XÂY DỰNG MÔ HÌNH HỌC MÁY

In [ ]:
#Chau muc_2
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
x=df.drop(['stroke'],axis=1)
y=df['stroke']
de_model = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = de_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Độ chính xác của mô hình Decision Tree là: ", de_model.score(X_test, y_test)*100,"%")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(de_model, x_data, y_data):
    # Tính toán dự đoán
    y_pred = de_model.predict(x_data)
    # Lấy các nhãn duy nhất
    labels = np.unique(y_data)
    # Tính toán ma trận nhầm lẫn
    cm = confusion_matrix(y_data, y_pred, labels=labels)
    # Vẽ ma trận nhầm lẫn
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='pink', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Nhãn dự đoán')
    plt.ylabel('Nhãn thực tế')
    plt.title('Ma trận nhầm lẫn')
    plt.show()

# Gọi hàm plot_confusion_matrix với mô hình logistic_model, dữ liệu x_train và nhãn y_train
plot_confusion_matrix(de_model, X_train, y_train)


In [ ]:
#Chau_muc2
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình Random Forest
rf = RandomForestClassifier(random_state=42)
# Tạo lưới tham số để tìm kiếm
param_dist = {'n_estimators': [100, 200, 300],'max_depth': [5, 10, 15, None],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4],'bootstrap': [True, False]}

random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_

# Tạo mô hình với siêu tham số tốt nhất
best_rf = RandomForestClassifier(**best_params, random_state=42)
# Huấn luyện mô hình
best_rf.fit(X_train, y_train)
# Dự đoán trên tập kiểm tra
y_pred = best_rf.predict(X_test)
# Đánh giá mô hình
print(classification_report(y_test, y_pred))
print("Độ chính xác của mô hình Random Forest là: ", best_rf.score(X_test, y_test)*100,"%")
# Vẽ ma trận nhầm lẫn
def plot_confusion_matrix(best_rf, X_test, y_test):
    # Dự đoán trên tập kiểm tra
    y_pred = best_rf.predict(X_test)
    # Tính toán ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred)
    # Vẽ biểu đồ
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
    plt.xlabel('Nhãn dự đoán')
    plt.ylabel('Nhãn thực tế')
    plt.title('Ma trận nhầm lẫn')
    plt.show()
# Gọi hàm để vẽ ma trận nhầm lẫn
plot_confusion_matrix(best_rf, X_test, y_test)


In [ ]:
#Chau_muc2
#vẽ features importance
import matplotlib.pyplot as plt
# Quan trọng tính của các đặc trưng
importances = best_rf.feature_importances_
data =pd.DataFrame( {'feature': ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'],'importance': best_rf.feature_importances_} )
# Sắp xếp theo thứ tự giảm dần của quan trọng tính
data = data.sort_values('importance', ascending=False)
print (data)
# Vẽ biểu đồ cột
plt.figure(figsize=(10, 6))
plt.barh(data['feature'], data['importance'])
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title('Features Importances')
# Đảo ngược trục y để đặc trưng quan trọng nhất ở trên
plt.gca().invert_yaxis()
plt.show()


In [ ]:
#Giang_Muc2

#import thu vien
import numpy as np
import seaborn as sns # mở cmd cài:  pip install seaborn
import matplotlib.pyplot as plt # mở cmd cài:  pip install matplotlib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# #Train căn bản
# from sklearn.linear_model import LogisticRegression

# clf = LogisticRegression(random_state= 10000)

# clf.fit(X_train, y_train)

# #Dự đoán
# from sklearn.metrics import classification_report, confusion_matrix

# # Dự đoán trên tập dữ liệu kiểm thử
# y_pred = clf.predict(X_test)

# # In báo cáo phân loại
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

In [ ]:
#chi
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Giả sử df là DataFrame chứa dữ liệu của bạn
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']

# Tăng cường dữ liệu bằng SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Khởi tạo mô hình Random Forest
rf = RandomForestClassifier(random_state=42)

# Tạo lưới tham số để tìm kiếm
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Sử dụng GridSearchCV để tìm kiếm siêu tham số tốt nhất
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

# Tạo mô hình với siêu tham số tốt nhất
best_rf = RandomForestClassifier(**best_params, random_state=42)

# Huấn luyện mô hình
best_rf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = best_rf.predict(X_test)

# Đánh giá mô hình
print(classification_report(y_test, y_pred))
print("Độ chính xác của mô hình Random Forest là: ", best_rf.score(X_test, y_test)*100, "%")

# Vẽ ma trận nhầm lẫn
def plot_confusion_matrix(best_rf, X_test, y_test):
    # Dự đoán trên tập kiểm tra
    y_pred = best_rf.predict(X_test)
    # Tính toán ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred)
    # Vẽ biểu đồ
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
    plt.xlabel('Nhãn dự đoán')
    plt.ylabel('Nhãn thực tế')
    plt.title('Ma trận nhầm lẫn')
    plt.show()

# Gọi hàm để vẽ ma trận nhầm lẫn
plot_confusion_matrix(best_rf, X_test, y_test)



In [ ]:
#chi
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from skopt import BayesSearchCV
import numpy as np

# Giả sử df là DataFrame chứa dữ liệu của bạn
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']

# Tăng cường dữ liệu bằng SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Khởi tạo mô hình Random Forest
rf = RandomForestClassifier(random_state=42)

# Sử dụng Bayesian Optimization để tìm kiếm siêu tham số tốt nhất
param_dist = {
    'n_estimators': (100, 1000),
    'max_depth': (5, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4),
    'bootstrap': [True, False]
}

bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_dist, n_iter=100, cv=5, n_jobs=-1, scoring='accuracy', random_state=42)
bayes_search.fit(X_train, y_train)
best_params = bayes_search.best_params_

# Tạo mô hình với siêu tham số tốt nhất
best_rf = RandomForestClassifier(**best_params, random_state=42)

# Huấn luyện mô hình
best_rf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = best_rf.predict(X_test)

# Đánh giá mô hình
print(classification_report(y_test, y_pred))
print("Độ chính xác của mô hình Random Forest là: ", best_rf.score(X_test, y_test)*100, "%")

# Vẽ ma trận nhầm lẫn
def plot_confusion_matrix(best_rf, X_test, y_test):
    # Dự đoán trên tập kiểm tra
    y_pred = best_rf.predict(X_test)
    # Tính toán ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred)
    # Vẽ biểu đồ
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
    plt.xlabel('Nhãn dự đoán')
    plt.ylabel('Nhãn thực tế')
    plt.title('Ma trận nhầm lẫn')
    plt.show()

# Gọi hàm để vẽ ma trận nhầm lẫn
plot_confusion_matrix(best_rf, X_test, y_test)
